In [1]:
if 'google.colab' in str(get_ipython()):
    print("Running on Colab")
    from google.colab import drive, output
    drive.mount('/content/drive')
    import sys
    !pip install torchaudio
    !pip install pythae
    output.clear()
    #%pdb on

    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
else:
    print("Running local")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform
import pythae
from pythae.pipelines import TrainingPipeline
from pythae.models import VQVAE, VQVAEConfig
from pythae.models import VAE, VAEConfig
from pythae.models import VAE_LinNF, VAE_LinNF_Config
from pythae.models import AE, AEConfig
from pythae.models import MIWAE, MIWAEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.models.nn import BaseDecoder
from pythae.models.base.base_utils import ModelOutput
from pythae.models.nn import BaseEncoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from Pythae_DataLoader import SoundscapeData
# from Pythae_Models import My_encoder
# from Pythae_Models import My_decoder


/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project


In [2]:
class My_encoder(BaseEncoder):

    def __init__(self,args):
        BaseEncoder.__init__(self)
        self.encoder = nn.Sequential(
            nn.Conv2d(1, args.latent_dim // 8, kernel_size=8, stride=3, padding=0),  # N, 256, 127, 8004
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 8, args.latent_dim // 4, kernel_size=8, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 4, args.latent_dim // 2, kernel_size=4, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 2, args.latent_dim, kernel_size=2, stride=2, padding=0),  # N, 512, 125,969
            nn.ReLU()
        ) 

    def forward(self, x: torch.Tensor):
        embedding = self.encoder(x)
        output = ModelOutput(
            embedding=embedding,
            # log_covariance=log_var # for VAE based models
        )
        return output


class My_decoder(BaseDecoder):

    def __init__(self, args):
        BaseDecoder.__init__(self)
        self.decoder = nn.Sequential(  # This is like go in opposite direction respect the encoder
            nn.ConvTranspose2d(args.latent_dim, args.latent_dim // 2, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 2, args.latent_dim // 4, kernel_size=4, stride=3, padding=0,
                               output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 4, args.latent_dim // 8, kernel_size=8, stride=3, padding=0,
                               output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 8, 1, kernel_size=8, stride=3, padding=0, output_padding=0),
            nn.Sigmoid()
        )

    def forward(self, z: torch.Tensor):
           reconstruction = self.decoder(z)
           output = ModelOutput(
                reconstruction=reconstruction
            )
           return output


In [3]:
my_training_config = BaseTrainerConfig(
    output_dir='my_model_AE',
    num_epochs=3,
    learning_rate=1e-3,
    batch_size=12,
    steps_saving=1
)

In [4]:
my_vae_config = model_config = AEConfig(
    input_dim=(1, 515, 515),
    latent_dim=64,
    uses_default_decoder= False,
    uses_default_encoder= False

)



In [5]:
encoder = My_encoder(model_config)
decoder= My_decoder(model_config)

In [6]:
my_vae_model = AE(
    model_config=my_vae_config,
    encoder = encoder,
    decoder = decoder
).to(device)

In [7]:
pipeline = TrainingPipeline(
    training_config=my_training_config,
    model=my_vae_model
)

In [8]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'


dataset = SoundscapeData(root_path, audio_length=12, ext="wav", win_length=1028)
dataset_pythae, dataset_big_test = random_split(dataset,
                                           [round(len(dataset)*0.3), len(dataset) - round(len(dataset)*0.3)], 
                                           generator=torch.Generator().manual_seed(1024))
dataset_train, dataset_test = random_split(dataset_pythae,
                                           [round(len(dataset_pythae)*0.7), len(dataset_pythae) - round(len(dataset_pythae)*0.7)], 
                                           generator=torch.Generator().manual_seed(1024))

In [9]:
iterator = iter(dataset_train)
dictado = next(iterator)
type(dictado.data)
dictado.data.shape

torch.Size([1, 515, 515])

In [10]:
pipeline(
    train_data=dataset_train, # must be torch.Tensor or np.array
    eval_data=dataset_test# must be torch.Tensor or np.array
)


Checking train dataset...
INFO:pythae.pipelines.training:Checking train dataset...
Checking eval dataset...
INFO:pythae.pipelines.training:Checking eval dataset...
Using Base Trainer

INFO:pythae.pipelines.training:Using Base Trainer

Model passed sanity check !

INFO:pythae.trainers.base_trainer.base_trainer:Model passed sanity check !

Created my_model_AE/AE_training_2022-10-23_19-10-59. 
Training config, checkpoints and final model will be saved here.

INFO:pythae.trainers.base_trainer.base_trainer:Created my_model_AE/AE_training_2022-10-23_19-10-59. 
Training config, checkpoints and final model will be saved here.

Successfully launched training !

INFO:pythae.trainers.base_trainer.base_trainer:Successfully launched training !



Training of epoch 1/3:   0%|          | 0/352 [00:00<?, ?batch/s]

Eval of epoch 1/3:   0%|          | 0/151 [00:00<?, ?batch/s]

Saved checkpoint at epoch 1

INFO:pythae.trainers.base_trainer.base_trainer:Saved checkpoint at epoch 1

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 60542.8289
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 60542.8289
Eval loss: 50847.2993
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 50847.2993
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 2/3:   0%|          | 0/352 [00:00<?, ?batch/s]

Eval of epoch 2/3:   0%|          | 0/151 [00:00<?, ?batch/s]

Saved checkpoint at epoch 2

INFO:pythae.trainers.base_trainer.base_trainer:Saved checkpoint at epoch 2

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 50932.8763
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 50932.8763
Eval loss: 50080.1685
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 50080.1685
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 3/3:   0%|          | 0/352 [00:00<?, ?batch/s]

Eval of epoch 3/3:   0%|          | 0/151 [00:00<?, ?batch/s]

Saved checkpoint at epoch 3

INFO:pythae.trainers.base_trainer.base_trainer:Saved checkpoint at epoch 3

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 50553.8634
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 50553.8634
Eval loss: 50321.6345
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 50321.6345
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Training ended!
INFO:pythae.trainers.base_trainer.base_trainer:Training ended!
Saved final model in my_model_AE/AE_training_2022-10-23_19-10-59/final_model
INFO:pythae.trainers.base_trainer.base_trainer:Saved final model in my_model_AE/AE_training_2022-10-23_19-10-59/final_model


## Using the trained model and samplers

In [11]:
import os
from pythae.models import AutoModel

In [12]:
last_training = sorted(os.listdir("my_model"))[-1]
trained_model = AutoModel.load_from_folder(os.path.join("my_model", last_training, "final_model"))

FileNotFoundError: ignored

## Using wandb logging

In [ ]:
from pythae.trainers.training_callbacks import WandbCallback

callbacks = [] # the TrainingPipeline expects a list of callbacks

wandb_cb = WandbCallback() # Build the callback 

# SetUp the callback 
wandb_cb.setup(
    training_config=training_config, # training config
    model_config=model_config, # model config
    project_name="your_wandb_project", # specify your wandb project
    entity_name="your_wandb_entity", # specify your wandb entity
)

callbacks.append(wandb_cb) # Add it to the callbacks list